# OpenAI Compatible API

SGLang provides an OpenAI compatible API for smooth transition from OpenAI services. Full reference of the API is available at [OpenAI API Reference](https://platform.openai.com/docs/api-reference).

This tutorial aims at these popular APIs:

- `chat/completions`
- `completions`
- `batches`
- `embeddings`(refer to [embedding_model.ipynb](embedding_model.ipynb))

## Chat Completions

### Usage

Similar to [send_request.ipynb](send_request.ipynb), we can send a chat completion request to SGLang server with OpenAI API format.

In [1]:
from sglang.utils import lauch_sglang_server, wait_for_server, terminate_process, highlight_text

server_process = lauch_sglang_server(
    command="python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 --host 0.0.0.0"
)

wait_for_server("http://localhost:30000")

highlight_text("Server is ready. Proceeding with the next steps.")

/home/chenyang/miniconda3/envs/AlphaMeemory/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
[2024-10-28 00:51:45] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Meta-Llama-3.1-8B-Instruct', chat_template=None, is_embedding=False, host='0.0.0.0', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, tp_size=1, stream_interval=1, random_seed=230242831, constrained_json_whitespace_pattern=None, log_level='i

INFO 10-28 00:51:53 weight_utils.py:236] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.53it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.42it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:00<00:00,  3.48it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  3.07it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.98it/s]

[2024-10-28 00:51:55 TP0] Load weight end. type=LlamaForCausalLM, dtype=torch.bfloat16, avail mem=32.22 GB
[2024-10-28 00:51:55 TP0] Memory pool end. avail mem=4.60 GB
[2024-10-28 00:51:55 TP0] Capture cuda graph begin. This can take up to several minutes.
[2024-10-28 00:52:03 TP0] max_total_num_tokens=217512, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2024-10-28 00:52:03] INFO:     Started server process [1159897]
[2024-10-28 00:52:03] INFO:     Waiting for appl

[2024-10-28 00:52:04] INFO:     127.0.0.1:34180 - "GET /v1/models HTTP/1.1" 200 OK


In [2]:
import openai

# Always assign an api_key, even if not specified during server initialization.
# Setting an API key during server initialization is strongly recommended.

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# Chat completion example

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

highlight_text(f"Response: {response}")

[2024-10-28 00:52:04 TP0] Prefill batch. #new-seq: 1, #new-token: 49, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-28 00:52:04] INFO:     127.0.0.1:42914 - "GET /get_model_info HTTP/1.1" 200 OK


[2024-10-28 00:52:04 TP0] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 1, cache hit rate: 1.79%, token usage: 0.00, #running-req: 1, #queue-req: 0
[2024-10-28 00:52:04] The server is fired up and ready to roll!


[2024-10-28 00:52:04] INFO:     127.0.0.1:42924 - "POST /generate HTTP/1.1" 200 OK
[2024-10-28 00:52:05] INFO:     127.0.0.1:34188 - "POST /v1/chat/completions HTTP/1.1" 200 OK


[2024-10-28 00:52:05 TP0] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, gen throughput (token/s): 21.59, #queue-req: 0


### Parameters

The chat completions API accepts OpenAI Chat Completions API's parameters. Refer to [OpenAI Chat Completions API](https://platform.openai.com/docs/api-reference/chat/create) for more details.

Here is an example of a detailed chat completion request:

In [3]:
response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {
            "role": "system",
            "content": "You are a knowledgeable historian who provides concise responses.",
        },
        {"role": "user", "content": "Tell me about ancient Rome"},
        {
            "role": "assistant",
            "content": "Ancient Rome was a civilization centered in Italy.",
        },
        {"role": "user", "content": "What were their major achievements?"},
    ],
    temperature=0.3,  # Lower temperature for more focused responses
    max_tokens=100,  # Reasonable length for a concise response
    top_p=0.95,  # Slightly higher for better fluency
    stop=["\n\n"],  # Simple stop sequence
    presence_penalty=0.2,  # Mild penalty to avoid repetition
    frequency_penalty=0.2,  # Mild penalty for more natural language
    n=1,  # Single response is usually more stable
    seed=42,  # Keep for reproducibility
)

highlight_text(f"Response: {response}")

[2024-10-28 00:52:05 TP0] Prefill batch. #new-seq: 1, #new-token: 48, #cached-token: 28, cache hit rate: 21.97%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2024-10-28 00:52:05] INFO:     127.0.0.1:34188 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Completions

### Usage

Completions API is similar to Chat Completions API, but without the `messages` parameter.

In [4]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="List 3 countries and their capitals.",
    temperature=0,
    max_tokens=64,
    n=1,
    stop=None,
)

highlight_text(f"Response: {response}")

[2024-10-28 00:52:05 TP0] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 1, cache hit rate: 21.28%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 00:52:06 TP0] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, gen throughput (token/s): 38.46, #queue-req: 0


[2024-10-28 00:52:07] INFO:     127.0.0.1:34188 - "POST /v1/completions HTTP/1.1" 200 OK


### Parameters

The completions API accepts OpenAI Completions API's parameters.  Refer to [OpenAI Completions API](https://platform.openai.com/docs/api-reference/completions/create) for more details.

Here is an example of a detailed completions request:

In [5]:
response = client.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    prompt="Write a short story about a space explorer.",
    temperature=0.7,  # Moderate temperature for creative writing
    max_tokens=150,  # Longer response for a story
    top_p=0.9,  # Balanced diversity in word choice
    stop=["\n\n", "THE END"],  # Multiple stop sequences
    presence_penalty=0.3,  # Encourage novel elements
    frequency_penalty=0.3,  # Reduce repetitive phrases
    n=1,  # Generate one completion
    seed=123,  # For reproducible results
)

highlight_text(f"Response: {response}")

[2024-10-28 00:52:07 TP0] Prefill batch. #new-seq: 1, #new-token: 9, #cached-token: 1, cache hit rate: 20.53%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 00:52:07 TP0] Decode batch. #running-req: 1, #token: 15, token usage: 0.00, gen throughput (token/s): 40.88, #queue-req: 0
[2024-10-28 00:52:08 TP0] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 42.20, #queue-req: 0
[2024-10-28 00:52:09 TP0] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 42.08, #queue-req: 0
[2024-10-28 00:52:10 TP0] Decode batch. #running-req: 1, #token: 135, token usage: 0.00, gen throughput (token/s): 41.94, #queue-req: 0


[2024-10-28 00:52:10] INFO:     127.0.0.1:34188 - "POST /v1/completions HTTP/1.1" 200 OK


## Batches

We have implemented the batches API for chat completions and completions. You can upload your requests in `jsonl` files, create a batch job, and retrieve the results when the batch job is completed (which takes longer but costs less).

The batches APIs are:

- `batches`
- `batches/{batch_id}/cancel`
- `batches/{batch_id}`

Here is an example of a batch job for chat completions, completions are similar.


In [6]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = [
    {
        "custom_id": "request-1",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [
                {"role": "user", "content": "Tell me a joke about programming"}
            ],
            "max_tokens": 50,
        },
    },
    {
        "custom_id": "request-2",
        "method": "POST",
        "url": "/chat/completions",
        "body": {
            "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
            "messages": [{"role": "user", "content": "What is Python?"}],
            "max_tokens": 50,
        },
    },
]

input_file_path = "batch_requests.jsonl"

with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    file_response = client.files.create(file=f, purpose="batch")

batch_response = client.batches.create(
    input_file_id=file_response.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

highlight_text(f"Batch job created with ID: {batch_response.id}")

[2024-10-28 00:52:10] INFO:     127.0.0.1:42934 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-28 00:52:10] INFO:     127.0.0.1:42934 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-10-28 00:52:10 TP0] Prefill batch. #new-seq: 2, #new-token: 30, #cached-token: 50, cache hit rate: 35.06%, token usage: 0.00, #running-req: 0, #queue-req: 0


In [7]:
while batch_response.status not in ["completed", "failed", "cancelled"]:
    time.sleep(3)
    print(f"Batch job status: {batch_response.status}...trying again in 3 seconds...")
    batch_response = client.batches.retrieve(batch_response.id)

if batch_response.status == "completed":
    print("Batch job completed successfully!")
    print(f"Request counts: {batch_response.request_counts}")

    result_file_id = batch_response.output_file_id
    file_response = client.files.content(result_file_id)
    result_content = file_response.read().decode("utf-8")

    results = [
        json.loads(line) for line in result_content.split("\n") if line.strip() != ""
    ]

    for result in results:
        highlight_text(f"Request {result['custom_id']}:")
        highlight_text(f"Response: {result['response']}")

    highlight_text("Cleaning up files...")
    # Only delete the result file ID since file_response is just content
    client.files.delete(result_file_id)
else:
    highlight_text(f"Batch job failed with status: {batch_response.status}")
    if hasattr(batch_response, "errors"):
        highlight_text(f"Errors: {batch_response.errors}")

[2024-10-28 00:52:11 TP0] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 51.49, #queue-req: 0


Batch job status: validating...trying again in 3 seconds...
[2024-10-28 00:52:13] INFO:     127.0.0.1:42934 - "GET /v1/batches/batch_c4b5b1ab-ad97-4591-8ac9-5b922d050b7c HTTP/1.1" 200 OK
Batch job completed successfully!
Request counts: BatchRequestCounts(completed=2, failed=0, total=2)
[2024-10-28 00:52:13] INFO:     127.0.0.1:42934 - "GET /v1/files/backend_result_file-b2c4aad4-6afc-4522-a51b-088a87f51aad/content HTTP/1.1" 200 OK


[2024-10-28 00:52:13] INFO:     127.0.0.1:42934 - "DELETE /v1/files/backend_result_file-b2c4aad4-6afc-4522-a51b-088a87f51aad HTTP/1.1" 200 OK


It takes a while to complete the batch job. You can use these two APIs to retrieve the batch job status or cancel the batch job.

1. `batches/{batch_id}`: Retrieve the batch job status.
2. `batches/{batch_id}/cancel`: Cancel the batch job.

Here is an example to check the batch job status.

In [8]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(100):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

highlight_text(f"Created batch job with ID: {batch_job.id}")
highlight_text(f"Initial status: {batch_job.status}")

time.sleep(10)

max_checks = 5
for i in range(max_checks):
    batch_details = client.batches.retrieve(batch_id=batch_job.id)

    highlight_text(
        f"Batch job details (check {i+1} / {max_checks}) // ID: {batch_details.id} // Status: {batch_details.status} // Created at: {batch_details.created_at} // Input file ID: {batch_details.input_file_id} // Output file ID: {batch_details.output_file_id}"
    )
    highlight_text(
            f"<strong>Request counts: Total: {batch_details.request_counts.total} // Completed: {batch_details.request_counts.completed} // Failed: {batch_details.request_counts.failed}</strong>"
        )

    time.sleep(3)

[2024-10-28 00:52:13] INFO:     127.0.0.1:42936 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-28 00:52:13] INFO:     127.0.0.1:42936 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-10-28 00:52:13 TP0] Prefill batch. #new-seq: 29, #new-token: 870, #cached-token: 725, cache hit rate: 44.14%, token usage: 0.00, #running-req: 0, #queue-req: 0
[2024-10-28 00:52:14 TP0] Prefill batch. #new-seq: 71, #new-token: 2130, #cached-token: 1775, cache hit rate: 45.04%, token usage: 0.00, #running-req: 29, #queue-req: 0
[2024-10-28 00:52:14 TP0] Decode batch. #running-req: 100, #token: 3725, token usage: 0.02, gen throughput (token/s): 224.21, #queue-req: 0
[2024-10-28 00:52:15 TP0] Decode batch. #running-req: 100, #token: 7725, token usage: 0.04, gen throughput (token/s): 3544.38, #queue-req: 0
[2024-10-28 00:52:16 TP0] Decode batch. #running-req: 100, #token: 11725, token usage: 0.05, gen throughput (token/s): 3448.12, #queue-req: 0
[2024-10-28 00:52:18 TP0] Decode batch. #running-req: 100, #token: 15725, token usage: 0.07, gen throughput (token/s): 3362.33, #queue-req: 0
[2024-10-28 00:52:19 TP0] Decode batch. #running-req: 100, #token: 19725, token usage: 0.09, gen thr

[2024-10-28 00:52:23] INFO:     127.0.0.1:49124 - "GET /v1/batches/batch_2ecdf8b9-27f7-4be0-a9f0-5e51cc087f7d HTTP/1.1" 200 OK


[2024-10-28 00:52:24 TP0] Decode batch. #running-req: 100, #token: 35725, token usage: 0.16, gen throughput (token/s): 2987.36, #queue-req: 0
[2024-10-28 00:52:25 TP0] Decode batch. #running-req: 100, #token: 39725, token usage: 0.18, gen throughput (token/s): 2918.74, #queue-req: 0


[2024-10-28 00:52:26] INFO:     127.0.0.1:49124 - "GET /v1/batches/batch_2ecdf8b9-27f7-4be0-a9f0-5e51cc087f7d HTTP/1.1" 200 OK


[2024-10-28 00:52:27 TP0] Decode batch. #running-req: 100, #token: 43725, token usage: 0.20, gen throughput (token/s): 2858.18, #queue-req: 0
[2024-10-28 00:52:28 TP0] Decode batch. #running-req: 100, #token: 47725, token usage: 0.22, gen throughput (token/s): 2799.15, #queue-req: 0


[2024-10-28 00:52:29] INFO:     127.0.0.1:49124 - "GET /v1/batches/batch_2ecdf8b9-27f7-4be0-a9f0-5e51cc087f7d HTTP/1.1" 200 OK


[2024-10-28 00:52:30 TP0] Decode batch. #running-req: 100, #token: 51725, token usage: 0.24, gen throughput (token/s): 2740.04, #queue-req: 0


[2024-10-28 00:52:32] INFO:     127.0.0.1:49124 - "GET /v1/batches/batch_2ecdf8b9-27f7-4be0-a9f0-5e51cc087f7d HTTP/1.1" 200 OK


[2024-10-28 00:52:35] INFO:     127.0.0.1:49124 - "GET /v1/batches/batch_2ecdf8b9-27f7-4be0-a9f0-5e51cc087f7d HTTP/1.1" 200 OK


Here is an example to cancel a batch job.

In [11]:
import json
import time
from openai import OpenAI

client = OpenAI(base_url="http://127.0.0.1:30000/v1", api_key="None")

requests = []
for i in range(500):
    requests.append(
        {
            "custom_id": f"request-{i}",
            "method": "POST",
            "url": "/chat/completions",
            "body": {
                "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
                "messages": [
                    {
                        "role": "system",
                        "content": f"{i}: You are a helpful AI assistant",
                    },
                    {
                        "role": "user",
                        "content": "Write a detailed story about topic. Make it very long.",
                    },
                ],
                "max_tokens": 500,
            },
        }
    )

input_file_path = "batch_requests.jsonl"
with open(input_file_path, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

with open(input_file_path, "rb") as f:
    uploaded_file = client.files.create(file=f, purpose="batch")

batch_job = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
)

highlight_text(f"Created batch job with ID: {batch_job.id}")
highlight_text(f"Initial status: {batch_job.status}")

time.sleep(10)

try:
    cancelled_job = client.batches.cancel(batch_id=batch_job.id)
    highlight_text(f"Cancellation initiated. Status: {cancelled_job.status}")
    assert cancelled_job.status == "cancelling"

    # Monitor the cancellation process
    while cancelled_job.status not in ["failed", "cancelled"]:
        time.sleep(3)
        cancelled_job = client.batches.retrieve(batch_job.id)
        highlight_text(f"Current status: {cancelled_job.status}")

    # Verify final status
    assert cancelled_job.status == "cancelled"
    highlight_text("Batch job successfully cancelled")

except Exception as e:
    highlight_text(f"Error during cancellation: {e}")
    raise e

finally:
    try:
        del_response = client.files.delete(uploaded_file.id)
        if del_response.deleted:
            highlight_text("Successfully cleaned up input file")
    except Exception as e:
        highlight_text(f"Error cleaning up: {e}")
        raise e

[2024-10-28 00:54:17] INFO:     127.0.0.1:57658 - "POST /v1/files HTTP/1.1" 200 OK
[2024-10-28 00:54:17] INFO:     127.0.0.1:57658 - "POST /v1/batches HTTP/1.1" 200 OK


[2024-10-28 00:54:17 TP0] Prefill batch. #new-seq: 109, #new-token: 109, #cached-token: 5886, cache hit rate: 74.75%, token usage: 0.01, #running-req: 0, #queue-req: 0
[2024-10-28 00:54:17 TP0] Prefill batch. #new-seq: 332, #new-token: 332, #cached-token: 17928, cache hit rate: 80.04%, token usage: 0.06, #running-req: 109, #queue-req: 59
[2024-10-28 00:54:19 TP0] Decode batch. #running-req: 441, #token: 22957, token usage: 0.11, gen throughput (token/s): 410.37, #queue-req: 59
[2024-10-28 00:54:22 TP0] Decode batch. #running-req: 441, #token: 40597, token usage: 0.19, gen throughput (token/s): 5187.40, #queue-req: 59
[2024-10-28 00:54:26 TP0] Decode batch. #running-req: 441, #token: 58237, token usage: 0.27, gen throughput (token/s): 4984.56, #queue-req: 59


[2024-10-28 00:54:27] INFO:     127.0.0.1:41908 - "POST /v1/batches/batch_bce8c1f8-a007-402a-b19a-ae280104214d/cancel HTTP/1.1" 200 OK


[2024-10-28 00:54:30] INFO:     127.0.0.1:41908 - "GET /v1/batches/batch_bce8c1f8-a007-402a-b19a-ae280104214d HTTP/1.1" 200 OK


[2024-10-28 00:54:30] INFO:     127.0.0.1:41908 - "DELETE /v1/files/backend_input_file-4d1d44ff-5efe-4aa8-9263-9df61d9ccd88 HTTP/1.1" 200 OK


In [46]:
terminate_process(server_process)

[2024-10-28 00:42:44] INFO:     Shutting down
[2024-10-28 00:42:45] INFO:     Waiting for application shutdown.
[2024-10-28 00:42:45] INFO:     Application shutdown complete.
[2024-10-28 00:42:45] INFO:     Finished server process [1154958]
W1028 00:42:45.164000 140548813764160 torch/_inductor/compile_worker/subproc_pool.py:126] SubprocPool unclean exit
